## Mars Coverage & Merchandising Data

Mars Coverage & Merchandising - User Check in and Reason for Non Adherecne

In [1]:
import pandas as pd
import pyodbc

# SQL Server connection details
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

# Create connection
conn = pyodbc.connect(conn_str)

### Store Coverage Data

In [2]:
coveragequery="""
WITH cte AS (
    SELECT
        UserName,
        LISStoreCode,
        -- Safe Visit Date conversion
        TRY_CONVERT(date, LEFT(VisitDate, 10), 105) AS VisitDate,
        AttendanceType,
        PJPStatus,
        -- Time conversions
        TRY_CONVERT(time, [CheckIn-Time])  AS CheckInTime,
        TRY_CONVERT(time, [CheckOut-Time]) AS CheckOutTime,
        TRY_CONVERT(time, TimeDuration)    AS TimeDuration,
        -- Covered logic
        CASE 
            WHEN TRY_CONVERT(date, LEFT(VisitDate, 10), 105) IS NOT NULL THEN 1
            ELSE 0
        END AS Covered,
        -- Merchandised flag
        CASE 
            WHEN AttendanceType = 'Attendance' THEN 1
            ELSE 0
        END AS Merchandised,
        -- Reason / Final Status
        CASE 
            WHEN AttendanceType = 'Attendance'
                THEN 'Merchandised'
            ELSE NoPermissionReason
        END AS Reason,
        ROW_NUMBER() OVER (
            PARTITION BY 
                UserName,
                LISStoreCode,
                TRY_CONVERT(date, LEFT(VisitDate, 10), 105)
            ORDER BY 
                TRY_CONVERT(time, [CheckIn-Time]) DESC
        ) AS rn

    FROM PowerBI_LISV2.MarsManagement.ISPStoreAttendance_NormalExport
    WHERE [month] = 12
      AND [year] = 2025
)
SELECT * FROM cte where rn=1;
    """

In [3]:
df_coverage = pd.read_sql(coveragequery, conn)

# Close connection
# conn.close()

# Preview result
df_coverage.head()

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_45588\2328032912.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_coverage = pd.read_sql(coveragequery, conn)


UserName        LISStoreCode   VisitDate AttendanceType PJPStatus  \
0  ISPMARSGT-0001  1044225010_C031482  2025-12-06     Attendance   Planned   
1  ISPMARSGT-0001  1044225010_C031482  2025-12-13     Attendance   Planned   
2  ISPMARSGT-0001  1044225010_C031482  2025-12-27     Attendance   Planned   
3  ISPMARSGT-0001  1044225010_C031955  2025-12-02     Attendance   Planned   
4  ISPMARSGT-0001  1044225010_C031955  2025-12-09     Attendance   Planned   

  CheckInTime CheckOutTime TimeDuration  Covered  Merchandised        Reason  \
0    12:21:04     12:37:03     00:15:00        1             1  Merchandised   
1    12:04:37     12:12:42     00:08:00        1             1  Merchandised   
2    11:52:25     12:05:33     00:13:00        1             1  Merchandised   
3    15:17:17     15:25:56     00:08:00        1             1  Merchandised   
4    13:35:49     13:47:42     00:11:00        1             1  Merchandised   

   rn  
0   1  
1   1  
2   1  
3   1  
4   1

In [4]:
df_coverage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25006 entries, 0 to 25005
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   UserName        25006 non-null  object
 1   LISStoreCode    25006 non-null  object
 2   VisitDate       25006 non-null  object
 3   AttendanceType  25006 non-null  object
 4   PJPStatus       25006 non-null  object
 5   CheckInTime     25006 non-null  object
 6   CheckOutTime    25006 non-null  object
 7   TimeDuration    25006 non-null  object
 8   Covered         25006 non-null  int64 
 9   Merchandised    25006 non-null  int64 
 10  Reason          25006 non-null  object
 11  rn              25006 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 2.3+ MB


#### PJP Query ( Planned )

In [5]:
pjpquery = """
SELECT distinct
    PJP.ISPStoreCode,PJP.UserName,PJP.Date
FROM PowerBI_LISV2.LISReporting.StoreUserMapping AS PJP
WHERE PJP.Status='Yes' AND
    PJP.UserName NOT LIKE '%test%'
    AND PJP.UserName NOT LIKE '%SUP%'
    AND PJP.UserName IS NOT NULL;
"""

In [6]:
df_pjp = pd.read_sql(pjpquery, conn)
df_pjp.head()

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_45588\4049190622.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pjp = pd.read_sql(pjpquery, conn)


ISPStoreCode        UserName        Date
0  10329665_C0011  ISPMARSGT-0075  2025-12-06
1  10329665_C0011  ISPMARSGT-0075  2025-12-13
2  10329665_C0011  ISPMARSGT-0075  2025-12-20
3  10329665_C0011  ISPMARSGT-0075  2025-12-27
4  10329665_C0081  ISPMARSGT-0075  2025-12-01

In [7]:
QUERY="""SELECT K.UserName,K.LastWorkingDate
            FROM (
                SELECT UserName,TRY_CONVERT(DATE,LastWorkingDate,105) AS LastWorkingDate ,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.MarsManagement.UserMaster
                where UserName Not like '%temp%' and UserName not Like '%Audtior%' and UserName Not Like '%sup%' AND LastWorkingDate IS NOT NULL
            ) AS K
    WHERE RNK = 1
    
    """
lwd = pd.read_sql(QUERY, conn)
lwd.head()

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_45588\2184775480.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lwd = pd.read_sql(QUERY, conn)


UserName LastWorkingDate
0  ISPMARSGT-0008      2025-03-18
1  ISPMARSGT-0009      2025-03-25
2  ISPMARSGT-0012      2025-04-15
3  ISPMARSGT-0014      2025-05-31
4  ISPMARSGT-0017      2025-02-26

In [8]:
df_pjp_final = pd.merge(df_pjp,lwd,on ='UserName',how = 'left') 

In [9]:
df_pjp_final.head()

ISPStoreCode        UserName        Date LastWorkingDate
0  10329665_C0011  ISPMARSGT-0075  2025-12-06             NaN
1  10329665_C0011  ISPMARSGT-0075  2025-12-13             NaN
2  10329665_C0011  ISPMARSGT-0075  2025-12-20             NaN
3  10329665_C0011  ISPMARSGT-0075  2025-12-27             NaN
4  10329665_C0081  ISPMARSGT-0075  2025-12-01             NaN

In [10]:
df_pjp_final['lwdchekflag'] = (
    df_pjp_final['LastWorkingDate'].isna() |
    (df_pjp_final['Date'] <= df_pjp_final['LastWorkingDate'])
).astype(int)

In [11]:
df_pjp_final=df_pjp_final[df_pjp_final['lwdchekflag']==1]

In [12]:
df_pjp_final['PJPPlannedStatus'] = 'Planned'

#### Unplanned PJP ( from ISP Store Coverage Data)

In [13]:
df_coverage_unplanned = df_coverage[df_coverage['PJPStatus']=='UnPlanned']

In [14]:
df_coverage_unplanned = df_coverage_unplanned[['LISStoreCode','UserName','VisitDate']]

In [15]:
df_coverage_unplanned=df_coverage_unplanned.drop_duplicates()

In [16]:
df_coverage_unplanned['PJPPlannedStatus']='UnPlanned'

In [17]:
df_coverage_unplanned=df_coverage_unplanned.rename(columns = {'LISStoreCode':'ISPStoreCode','VisitDate':'Date'})

In [18]:
df_pjp_planned_unplanned_appened = pd.concat([df_pjp_final, df_coverage_unplanned], axis=0)

##### Combined PJP ( Planned & Unplanned)

In [19]:
df_pjp_planned_unplanned_appened.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29873 entries, 0 to 24955
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ISPStoreCode      29873 non-null  object 
 1   UserName          29873 non-null  object 
 2   Date              29873 non-null  object 
 3   LastWorkingDate   380 non-null    object 
 4   lwdchekflag       26366 non-null  float64
 5   PJPPlannedStatus  29873 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.6+ MB


#### Attendance Data ( User Attendance)

In [20]:
### Attendance Data ( MasterExport)

In [21]:
AttendanceQuery = """
SELECT UserName,TRY_CONVERT(DATE,Date,105)AS Date,
-- VisitDate,
[In-Time],[Out-Time],Status
FROM PowerBI_LISV2.MarsManagement.ISPAttendanceMasterExport
WHERE MONTH = 12 AND YEAR = 2025
"""
UserAttendance = pd.read_sql(AttendanceQuery, conn)
UserAttendance.head()

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_45588\4242686936.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  UserAttendance = pd.read_sql(AttendanceQuery, conn)


UserName        Date   In-Time  Out-Time            Status
0  ISPMARSGT-0066  2025-12-01  09:30:33  17:31:43           Present
1  ISPMARSGT-0039  2025-12-01  08:30:35  19:20:20           Present
2  ISPMARSGT-0107  2025-12-01  09:55:54  20:30:12           Present
3  ISPMARSGT-0097  2025-12-01      None      None             Leave
4  ISPMARSGT-0128  2025-12-01      None      None  Compensatory Off

In [22]:
UserAttendance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3236 entries, 0 to 3235
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   UserName  3236 non-null   object
 1   Date      3236 non-null   object
 2   In-Time   2342 non-null   object
 3   Out-Time  2342 non-null   object
 4   Status    3203 non-null   object
dtypes: object(5)
memory usage: 126.5+ KB


#### Combining PJP & Attend Data

In [23]:
ppj_attendance = pd.merge(df_pjp_planned_unplanned_appened,UserAttendance,on = ['UserName','Date'],how = 'left')
# ppj_attendance = ppj_attendance.drop(columns=['VisitDate'], inplace=True)

In [24]:
ppj_attendance.head()

ISPStoreCode        UserName        Date LastWorkingDate  lwdchekflag  \
0  10329665_C0011  ISPMARSGT-0075  2025-12-06             NaN          1.0   
1  10329665_C0011  ISPMARSGT-0075  2025-12-13             NaN          1.0   
2  10329665_C0011  ISPMARSGT-0075  2025-12-20             NaN          1.0   
3  10329665_C0011  ISPMARSGT-0075  2025-12-27             NaN          1.0   
4  10329665_C0081  ISPMARSGT-0075  2025-12-01             NaN          1.0   

  PJPPlannedStatus   In-Time  Out-Time   Status  
0          Planned  10:44:08  20:57:19  Present  
1          Planned  10:31:16  18:51:45  Present  
2          Planned      None      None    Leave  
3          Planned  10:22:24  23:45:00  Present  
4          Planned      None      None    Leave

#### Combining ppj_attendance & Coverage Data

In [25]:
coverage_pjp=pd.merge(ppj_attendance,df_coverage, left_on = ['ISPStoreCode','UserName','Date'],
         right_on=['LISStoreCode','UserName','VisitDate'], how = 'left')


In [26]:
coverage_pjp=coverage_pjp.drop(columns = ['LastWorkingDate','lwdchekflag','LISStoreCode','PJPStatus','rn'])

In [27]:
coverage_pjp.to_csv('coverage_pjp.csv')

In [28]:
coverage_pjp.head()

ISPStoreCode        UserName        Date PJPPlannedStatus   In-Time  \
0  10329665_C0011  ISPMARSGT-0075  2025-12-06          Planned  10:44:08   
1  10329665_C0011  ISPMARSGT-0075  2025-12-13          Planned  10:31:16   
2  10329665_C0011  ISPMARSGT-0075  2025-12-20          Planned      None   
3  10329665_C0011  ISPMARSGT-0075  2025-12-27          Planned  10:22:24   
4  10329665_C0081  ISPMARSGT-0075  2025-12-01          Planned      None   

   Out-Time   Status   VisitDate AttendanceType CheckInTime CheckOutTime  \
0  20:57:19  Present  2025-12-06   NoPermission    12:06:03     12:06:03   
1  18:51:45  Present  2025-12-13   NoPermission    12:13:13     12:13:13   
2      None    Leave         NaN            NaN         NaN          NaN   
3  23:45:00  Present  2025-12-27   NoPermission    11:59:40     11:59:40   
4      None    Leave         NaN            NaN         NaN          NaN   

  TimeDuration  Covered  Merchandised                    Reason  
0     00:00:00      1.0           0.0  Store Temporarily Closed  
1     00:00:00      1.0           0.0  Store Temporarily Closed  
2          NaN      NaN           NaN                       NaN  
3     00:00:00      1.0           0.0  Store Temporarily Closed  
4          NaN      NaN           NaN                       NaN

In [29]:
# SELECT
#     PJP.*,
#     cte.Covered,
#     cte.Merchandised,
#     cte.Reason,
#     cte.CheckInTime,
#     cte.CheckOutTime,
#     cte.TimeDuration
# FROM LISReporting.StoreUserMapping AS PJP
# Right JOIN cte
#     ON PJP.ISPStoreCode = cte.LISStoreCode
#    AND PJP.UserName    = cte.UserName
#    and PJP.Date=cte.VisitDate
#    AND cte.rn = 1   -- 👈 only latest visit per user-store-day
# WHERE 
#     PJP.UserName NOT LIKE '%test%'
#     AND PJP.UserName NOT LIKE '%SUP%'
#     AND PJP.UserName IS NOT NULL;